In [1]:
"""
Preprocessing based on: https://github.com/truongkhanhduy95/Heritage-Health-Prize
"""
import zipfile
from os import path
from urllib import request
from os.path import join as os_join
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split

from tqdm import tqdm as tqdm
import json
from os.path import join as os_join

res_path = '../results'
cache_path = '../cache'




In [6]:
res_path = '../results'
cache_path = '../cache'
data_path = '../data'

column_names = ['MemberID', 'ProviderID', 'Sex', 'AgeAtFirstClaim']#,'PrimaryConditionGroup','ProcedureGroup','Specialty']
# column_names = [  'Sex', 'AgeAtFirstClaim']

claims_cat_names = ['PrimaryConditionGroup', 'Specialty', 'ProcedureGroup', 'PlaceSvc']

health_file = os_join(data_path,'health','health_full.csv')

df_health = pd.read_csv(health_file, sep=',')
df_health
labels =  df_health['max_CharlsonIndex']>0

features = df_health.drop('max_CharlsonIndex', axis=1)

continuous_vars = [col for col in features.columns if '=' not in col]
continuous_columns = [features.columns.get_loc(var) for var in continuous_vars]
# print (continuous_columns)
# self.protected_unique = 2
# protected = np.logical_or(
#     features['AgeAtFirstClaim=60-69'], np.logical_or(
#         features['AgeAtFirstClaim=70-79'], features['AgeAtFirstClaim=80+']
#     )
# )

one_hot_columns = {}
for column_name in column_names:
    ids = [i for i, col in enumerate(features.columns) if col.startswith('{}='.format(column_name))]
    # print (ids, column_name)
    if len(ids) > 0:
        assert len(ids) == ids[-1] - ids[0] + 1
        one_hot_columns[column_name] = [features.columns[_] for _ in ids]

# print('categorical features: ', one_hot_columns)
# print(features.columns.to_list())

# for a, cols in one_hot_columns.items():
#     for _ in cols:
#         unique_val = features[_].unique()
#         print (_, unique_val.shape, unique_val)

features['label'] = labels
# print (one_hot_columns)
# print (continuous_vars)

all_one_hots = []
for cols in [one_hot_columns[_] for _ in one_hot_columns.keys()]:
    for col in cols:
        all_one_hots.append(col)
selected_col = continuous_vars + all_one_hots + ['label']
selected_col.remove('Sex=?')

print (all_one_hots)
print (selected_col)
dataset = features.loc[:,selected_col]
print(dataset.shape)
dataset = dataset[ ((dataset['Sex=F'] + dataset['Sex=M'])!=0) ]
dataset = dataset.drop(['Sex=F'], axis=1) 
print(dataset.shape)



from sklearn.preprocessing import MinMaxScaler
dataset[continuous_vars] = MinMaxScaler().fit_transform(dataset[continuous_vars])
print (dataset.columns)
dataset.describe()


['Sex=?', 'Sex=F', 'Sex=M', 'AgeAtFirstClaim=0-9', 'AgeAtFirstClaim=10-19', 'AgeAtFirstClaim=20-29', 'AgeAtFirstClaim=30-39', 'AgeAtFirstClaim=40-49', 'AgeAtFirstClaim=50-59', 'AgeAtFirstClaim=60-69', 'AgeAtFirstClaim=70-79', 'AgeAtFirstClaim=80+', 'AgeAtFirstClaim=?']
['LabCount_total', 'LabCount_months', 'DrugCount_total', 'DrugCount_months', 'no_Claims', 'no_Providers', 'no_Vendors', 'no_PCPs', 'PayDelay_total', 'PayDelay_max', 'PayDelay_min', 'Sex=F', 'Sex=M', 'AgeAtFirstClaim=0-9', 'AgeAtFirstClaim=10-19', 'AgeAtFirstClaim=20-29', 'AgeAtFirstClaim=30-39', 'AgeAtFirstClaim=40-49', 'AgeAtFirstClaim=50-59', 'AgeAtFirstClaim=60-69', 'AgeAtFirstClaim=70-79', 'AgeAtFirstClaim=80+', 'AgeAtFirstClaim=?', 'label']
(218415, 24)
(177266, 23)
Index(['LabCount_total', 'LabCount_months', 'DrugCount_total',
       'DrugCount_months', 'no_Claims', 'no_Providers', 'no_Vendors',
       'no_PCPs', 'PayDelay_total', 'PayDelay_max', 'PayDelay_min', 'Sex=M',
       'AgeAtFirstClaim=0-9', 'AgeAtFirstCla

,LabCount_total,LabCount_months,DrugCount_total,DrugCount_months,no_Claims,no_Providers,no_Vendors,no_PCPs,PayDelay_total,PayDelay_max,...,AgeAtFirstClaim=0-9,AgeAtFirstClaim=10-19,AgeAtFirstClaim=20-29,AgeAtFirstClaim=30-39,AgeAtFirstClaim=40-49,AgeAtFirstClaim=50-59,AgeAtFirstClaim=60-69,AgeAtFirstClaim=70-79,AgeAtFirstClaim=80+,AgeAtFirstClaim=?
count,177266.000000,177266.000000,177266.000000,177266.000000,177266.000000,177266.000000,177266.000000,177266.000000,177266.000000,177266.000000,...,177266.000000,177266.000000,177266.000000,177266.000000,177266.000000,177266.000000,177266.000000,177266.000000,177266.000000,177266.000000
mean,0.053793,0.127815,0.110869,0.288062,0.242654,0.150186,0.163506,0.255272,0.074055,0.505224,...,0.093577,0.093453,0.061659,0.103302,0.143547,0.119386,0.121964,0.159478,0.068671,0.034964
std,0.067063,0.155437,0.176186,0.334350,0.241870,0.116350,0.115268,0.036590,0.083084,0.303973,...,0.291240,0.291067,0.240535,0.304354,0.350631,0.324243,0.327245,0.366122,0.252894,0.183690
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.068182,0.074074,0.095238,0.250000,0.018542,0.259259,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.036036,0.083333,0.023810,0.083333,0.159091,0.111111,0.142857,0.250000,0.044663,0.419753,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.081081,0.166667,0.142857,0.500000,0.318182,0.185185,0.238095,0.250000,0.097388,0.746914,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
dataset[['label']].to_numpy()

array([[ True],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [10]:
import pickle

def split_set(np_data,train_ratio, RS):
    """Split feature-label matrix into train/dev/test"""
    X = np_data[:,:-1].astype(float)
    Y = np_data[:,-1].astype(int).reshape(-1,1)
    X_train, X_rest, y_train, y_rest = train_test_split(X, Y, test_size=(1.0-train_ratio), random_state=RS)
    X_dev, X_test, y_dev, y_test = train_test_split(X_rest, y_rest, test_size=0.5, random_state=RS)
    return X_train, y_train, X_dev, y_dev, X_test, y_test


input_feature_list = [ 'Sex=M', 'LabCount_total', 'LabCount_months', 'DrugCount_total',
                    'DrugCount_months', 'no_Claims', 'no_Providers', 'no_Vendors',
                    'no_PCPs', 'PayDelay_total', 'PayDelay_max', 'PayDelay_min',
                    'AgeAtFirstClaim=0-9', 'AgeAtFirstClaim=10-19', 'AgeAtFirstClaim=20-29',
                    'AgeAtFirstClaim=30-39', 'AgeAtFirstClaim=40-49',
                    'AgeAtFirstClaim=50-59', 'AgeAtFirstClaim=60-69',
                    'AgeAtFirstClaim=70-79', 'AgeAtFirstClaim=80+', 'AgeAtFirstClaim=?',]
Y = dataset[['label']].to_numpy()
X = dataset[input_feature_list].to_numpy()


data_set = np.hstack([X,Y])
for random_seed in range(10):
    RS = np.random.RandomState(random_seed)
    # train/dev/test set
    train_ratio = 0.7 # dev and test share the rest
    X_train, y_train, X_dev, y_dev, X_test, y_test = split_set(data_set,train_ratio, RS)
    print ('Train feature/label shape:',X_train.shape, y_train.shape)
    print ('Dev. feature/label shape:',X_dev.shape, y_dev.shape)
    print ('Test feature/label shape:',X_test.shape, y_test.shape)

    data_output = {
        "X_train":X_train,
        "y_train":y_train,
        "X_dev":X_dev,
        "y_dev":y_dev,
        "X_test":X_test,
        "y_test":y_test,
    }
    cache_path = '/Users/xingzhiguo/Documents/git_project/NN-verification/cache'
    cache_file_path = os_join(cache_path,f'np-health-data-rs={random_seed}.pkl')
    with open (cache_file_path,'wb') as f:
        pickle.dump(data_output,f)
    print (f'saved data matrix to {cache_file_path}')



Train feature/label shape: (124086, 22) (124086, 1)
Dev. feature/label shape: (26590, 22) (26590, 1)
Test feature/label shape: (26590, 22) (26590, 1)
saved data matrix to /Users/xingzhiguo/Documents/git_project/NN-verification/cache/np-health-data-rs=0.pkl
Train feature/label shape: (124086, 22) (124086, 1)
Dev. feature/label shape: (26590, 22) (26590, 1)
Test feature/label shape: (26590, 22) (26590, 1)
saved data matrix to /Users/xingzhiguo/Documents/git_project/NN-verification/cache/np-health-data-rs=1.pkl
Train feature/label shape: (124086, 22) (124086, 1)
Dev. feature/label shape: (26590, 22) (26590, 1)
Test feature/label shape: (26590, 22) (26590, 1)
saved data matrix to /Users/xingzhiguo/Documents/git_project/NN-verification/cache/np-health-data-rs=2.pkl
Train feature/label shape: (124086, 22) (124086, 1)
Dev. feature/label shape: (26590, 22) (26590, 1)
Test feature/label shape: (26590, 22) (26590, 1)
saved data matrix to /Users/xingzhiguo/Documents/git_project/NN-verification/ca